In [ ]:
#default_exp augmentations

## augmentations
Implmentation heavily inspired by Ignacio Oguiza - oguiza@gmail.com

In [ ]:
import numpy as np
import random
from functools import partial
from scipy.interpolate import CubicSpline
import torch

In [ ]:
#exports
def random_curve_generator(ts, magnitude=.1, order=4, noise=None):
    seq_len = ts.shape[-1]
    x = np.linspace(-seq_len, 2 * seq_len - 1, 3 * (order - 1) + 1, dtype=int)
    x2 = np.random.normal(loc=1.0, scale=magnitude, size=len(x))
    f = CubicSpline(x, x2, axis=-1)
    return f(np.arange(seq_len))


def random_cum_curve_generator(ts, magnitude=.1, order=4, noise=None):
    x = random_curve_generator(ts, magnitude=magnitude, order=order, noise=noise).cumsum()
    x -= x[0]
    x /= x[-1]
    x = np.clip(x, 0, 1)
    return x * (ts.shape[-1] - 1)


def random_cum_noise_generator(ts, magnitude=.1, noise=None):
    seq_len = ts.shape[-1]
    x = (np.ones(seq_len) + np.random.normal(loc=0, scale=magnitude, size=seq_len)).cumsum()
    x -= x[0]
    x /= x[-1]
    x = np.clip(x, 0, 1)
    return x * (ts.shape[-1] - 1)


def magnoise(x, magnitude=.1, add=True):
    if magnitude <= 0: return x
    y = None
    if isinstance(x, list):
        y = x[1]
        x = x[0]
    seq_len = x.shape[-1]
    noise = torch.normal(0, magnitude, (1, seq_len), dtype=x.dtype, device=x.device)
    if add:
        output = x + noise
        return output if y is None else [output, y]
    else:
        output = x * (1 + noise)
        return output if y is None else [output, y]


In [ ]:
magnoise(torch.tensor([2.1]))

tensor([[2.2296]])